## Preprocesamiento

In [34]:
!pip install requests

In [35]:
"""
Obtener lista de ligas de la API de fútbol.
"""
import requests
import pandas as pd
from pprint import pprint

# 1. Configuración
API_KEY = "4b0e225616f3a478f7816d98c6435b49"  # Reemplaza con tu clave real
BASE_URL = "https://v3.football.api-sports.io"

@st.cache_data
def obtener_ligas_dataframe(api_key):
    """
    Consulta la API de fútbol y devuelve las ligas en un DataFrame de Pandas.
    """
    url = f"{BASE_URL}/leagues"
    headers = {'x-apisports-key': api_key}

    try:
        print("⏳ Conectando con la API...")
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Lanza error si el status no es 200

        data = response.json()

        # Verificar si hay errores lógicos de la API (ej. key inválida)
        if data.get('errors'):
            print("🚨 La API devolvió un error lógico:")
            pprint(data['errors'])
            return None

        results = data['response']
        print(f"✅ Datos recibidos. Procesando {len(results)} registros...")

        # 2. Convertir a DataFrame usando json_normalize
        # Esto aplana la estructura: 'league': {'name': 'x'} -> 'league.name': 'x'
        df = pd.json_normalize(results)

        # 3. Selección y renombrado de columnas (Opcional, pero recomendado para limpieza)
        # La API devuelve mucha info (logo, temporadas, etc.), aquí filtramos lo útil.
        columnas_utiles = [
            'league.id',
            'league.name',
            'league.type',
            'country.name',
            'country.code'
        ]

        # Filtramos si las columnas existen
        cols_existentes = [c for c in columnas_utiles if c in df.columns]
        df_final = df[cols_existentes].copy()

        # Renombramos para que sea más bonito (ej. league.name -> Liga)
        nombres_columnas = {
            'league.id': 'ID_Liga',
            'league.name': 'Nombre_Liga',
            'league.type': 'Tipo',
            'country.name': 'Pais',
            'country.code': 'Codigo_Pais'
        }
        df_final.rename(columns=nombres_columnas, inplace=True)

        return df_final

    except requests.exceptions.HTTPError as errh:
        print(f"🚨 Error HTTP: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"🚨 Error de Conexión: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"🚨 Error de Tiempo de espera: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"🚨 Error General: {err}")

    return None


# --- Ejecución del Script ---

if __name__ == "__main__":
    # Llamamos a la función
    df = obtener_ligas_dataframe(API_KEY)

    if df is not None:
        print("-" * 50)
        print("📊 VISTA PREVIA DEL DATAFRAME:")
        print("-" * 50)

        # Mostrar las primeras 10 filas
        print(df.head(10))

        # Mostrar información sobre las columnas y tipos de datos
        print("\nℹ️ Info del DataFrame:")
        print(df.info())

        # Opcional: Exportar a Excel o CSV
        df.to_csv('ligas_futbol.csv', index=False)
        print("\n📁 Archivo 'ligas_futbol.csv' guardado.")

2025-11-23 14:56:03.150 No runtime found, using MemoryCacheStorageManager
2025-11-23 14:56:03.152 No runtime found, using MemoryCacheStorageManager
2025-11-23 14:56:03.154 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:03.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:03.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:03.159 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


⏳ Conectando con la API...


2025-11-23 14:56:03.661 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:03.663 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:03.664 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:04.266 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:04.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 14:56:04.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ Datos recibidos. Procesando 1207 registros...
--------------------------------------------------
📊 VISTA PREVIA DEL DATAFRAME:
--------------------------------------------------
   ID_Liga         Nombre_Liga    Tipo         Pais Codigo_Pais
0        4   Euro Championship     Cup        World        None
1       21  Confederations Cup     Cup        World        None
2       62             Ligue 2  League       France          FR
3       61             Ligue 1  League       France          FR
4      144  Jupiler Pro League  League      Belgium          BE
5       71             Serie A  League       Brazil          BR
6       39      Premier League  League      England      GB-ENG
7       78          Bundesliga  League      Germany          DE
8      135             Serie A  League        Italy          IT
9       88          Eredivisie  League  Netherlands          NL

ℹ️ Info del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1207 entries, 0 to 1206
Data columns (tota

In [36]:
"""

Temporada de los equipos

"""

import requests
import pandas as pd
from pprint import pprint

# 1. Configuración
# ⚠️ IMPORTANTE: He ocultado la clave por seguridad.
# Vuelve a poner tu clave real aquí abajo.
API_KEY = "4b0e225616f3a478f7816d98c6435b49"
BASE_URL = "https://v3.football.api-sports.io"

def obtener_clasificacion(api_key, league_id, season):
    """
    Obtiene la tabla de posiciones y estadísticas de los equipos
    para una liga y temporada específica.
    """
    url = f"{BASE_URL}/standings"
    headers = {'x-apisports-key': api_key}
    # Parámetros obligatorios para este endpoint
    params = {
        'league': league_id,
        'season': season
    }

    try:
        print(f"⏳ Consultando datos para Liga ID: {league_id}, Temporada: {season}...")
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        data = response.json()

        if data.get('errors'):
            print("🚨 La API devolvió un error lógico:")
            pprint(data['errors'])
            return None

        # Verificar si hay resultados (a veces la liga no tiene datos para ese año)
        if data['results'] == 0:
            print("⚠️ No se encontraron datos para esta liga/temporada.")
            return None

        # La estructura de 'standings' es: response -> league -> standings -> [ [equipos] ]
        # Nota: Es una lista de listas porque algunas ligas tienen grupos.
        # Tomamos el primer grupo (índice 0) para ligas regulares.
        standings_data = data['response'][0]['league']['standings'][0]

        # Normalizamos los datos
        df = pd.json_normalize(standings_data)

        # Selección y limpieza de columnas más relevantes
        columnas_deseadas = {
            'rank': 'Posicion',
            'team.name': 'Equipo',
            'points': 'Puntos',
            'form': 'Forma',
            'all.played': 'PJ',      # Partidos Jugados
            'all.win': 'PG',         # Partidos Ganados
            'all.draw': 'PE',        # Partidos Empatados
            'all.lose': 'PP',        # Partidos Perdidos
            'all.goals.for': 'GF',   # Goles a Favor
            'all.goals.against': 'GC', # Goles en Contra
            'goalsDiff': 'Dif_Goles'
        }

        # Filtramos y renombramos
        df_final = df[columnas_deseadas.keys()].rename(columns=columnas_deseadas)

        return df_final

    except Exception as e:
        print(f"🚨 Error al procesar datos: {e}")
        return None

# --- Ejecución del Script ---

if __name__ == "__main__":

    # A. Parámetros de búsqueda
    # ID 39 = Premier League, 140 = La Liga, etc.
    # Debes sacar este ID del DataFrame de ligas que hiciste antes.
    ID_LIGA_BUSCAR = 22
    TEMPORADA = 2023

    # B. Llamada a la función
    df_equipos = obtener_clasificacion(API_KEY, ID_LIGA_BUSCAR, TEMPORADA)

    if df_equipos is not None:
        print("\n" + "="*50)
        print(f"⚽ TABLA DE POSICIONES (Liga {ID_LIGA_BUSCAR} - {TEMPORADA})")
        print("="*50)

        # Mostrar tabla
        print(df_equipos.to_string(index=False))

        # Exportar
        nombre_archivo = f"stats_equipos_{ID_LIGA_BUSCAR}_{TEMPORADA}.csv"
        df_equipos.to_csv(nombre_archivo, index=False)
        print(f"\n📁 Datos guardados en '{nombre_archivo}'")
    else:
        print("\n❌ No se pudo obtener la información.")


⏳ Consultando datos para Liga ID: 22, Temporada: 2023...

⚽ TABLA DE POSICIONES (Liga 22 - 2023)
 Posicion              Equipo  Puntos Forma  PJ  PG  PE  PP  GF  GC  Dif_Goles
        1                 USA       7   WWD   3   2   1   0  13   1         12
        2             Jamaica       7   WWD   3   2   1   0  10   2          8
        3 Trinidad and Tobago       3   LLW   3   1   0   2   4  10         -6
        4 St. Kitts and Nevis       0   LLL   3   0   0   3   0  14        -14

📁 Datos guardados en 'stats_equipos_22_2023.csv'


# Streamlit con API-football

para utilizar los datos en stream por medio de este framework

In [ ]:

ligas = pd.read_csv('ligas_futbol.csv')
ligas.describe(include='all')

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
# import requests  # Descomenta esto cuando uses la API real

# --- CONFIGURACIÓN DE LA PÁGINA ---
st.set_page_config(page_title="Dashboard Fútbol", layout="wide")

# --- TÍTULO Y ENCABEZADO ---
st.title("⚽ Analítica de Fútbol Pro")
st.markdown("Explora las estadísticas de los mejores jugadores de la temporada.")

# --- 1. FUNCIÓN DE CARGA DE DATOS ---
# Usamos @st.cache_data para que no llame a la API cada vez que cambias un filtro
@st.cache_data
def cargar_datos():
    # --- MODO DEMO (DATOS FALSOS PARA PROBAR AHORA) ---
    data = {
        'Jugador': ['Haaland', 'Salah', 'Son', 'Watkins', 'Saka', 'Isak', 'Foden', 'Palmer', 'Darwin'],
        'Equipo': ['Man City', 'Liverpool', 'Tottenham', 'Aston Villa', 'Arsenal', 'Newcastle', 'Man City', 'Chelsea', 'Liverpool'],
        'Goles': [27, 18, 17, 19, 16, 21, 19, 22, 11],
        'Asistencias': [5, 10, 10, 13, 9, 2, 8, 11, 8],
        'Partidos': [31, 32, 34, 37, 35, 30, 34, 33, 29]
    }
    df = pd.DataFrame(data)

    # --- MODO REAL (TU CÓDIGO DE API) ---
    # Si tuvieras la API Key, aquí iría el request:
    # url = "https://v3.football.api-sports.io/players/topscorers?league=39&season=2023"
    # headers = {'x-apisports-key': 'TU_KEY'}
    # response = requests.get(url, headers=headers).json()
    # ... lógica para crear el DataFrame ...

    return df

# Cargamos los datos
df = cargar_datos()

# --- 2. BARRA LATERAL (SIDEBAR) - FILTROS ---
st.sidebar.header("Filtros")

# Filtro por Equipo
equipos_disponibles = df['Equipo'].unique()
seleccion_equipos = st.sidebar.multiselect(
    "Selecciona Equipos:",
    options=equipos_disponibles,
    default=equipos_disponibles # Por defecto selecciona todos
)

# Filtro por Goles (Slider)
min_goles = st.sidebar.slider("Goles Mínimos:", 0, 30, 10)

# --- 3. APLICAR FILTROS AL DATAFRAME ---
# Filtramos el DF base según lo que eligió el usuario
df_filtrado = df[
    (df['Equipo'].isin(seleccion_equipos)) &
    (df['Goles'] >= min_goles)
]

# --- 4. MÉTRICAS PRINCIPALES (KPIs) ---
# Mostramos números grandes arriba
col1, col2, col3 = st.columns(3)
col1.metric("Jugadores Mostrados", len(df_filtrado))
col2.metric("Total Goles", df_filtrado['Goles'].sum())
col3.metric("Promedio Goles", round(df_filtrado['Goles'].mean(), 2))

st.divider() # Línea separadora

# --- 5. GRÁFICOS INTERACTIVOS ---
c1, c2 = st.columns((2, 1)) # La columna 1 es el doble de ancha que la 2

with c1:
    st.subheader("Goles vs Asistencias")
    fig_scatter = px.scatter(
        df_filtrado,
        x="Asistencias",
        y="Goles",
        color="Equipo",
        size="Partidos",
        hover_name="Jugador",
        text="Jugador",
        size_max=40
    )
    fig_scatter.update_traces(textposition='top center')
    st.plotly_chart(fig_scatter, use_container_width=True)

with c2:
    st.subheader("Goleadores por Equipo")
    # Agrupamos para ver qué equipo tiene más goles en total (de los filtrados)
    df_agrupado = df_filtrado.groupby('Equipo')['Goles'].sum().reset_index()
    fig_bar = px.bar(df_agrupado, x='Equipo', y='Goles', color='Equipo')
    st.plotly_chart(fig_bar, use_container_width=True)

# --- 6. TABLA DE DATOS ---
st.subheader("Datos Detallados")
st.dataframe(df_filtrado.set_index('Jugador'), use_container_width=True)